In [ ]:
%flink.ssql(type=update)

-- This table connects the stock data stream from Kinesis 
-- and defines a watermark for event-time processing.

DROP TABLE IF EXISTS stock_table;
CREATE TABLE stock_table(
    `date` STRING,
    ticker VARCHAR(6),
    open_price FLOAT,
    high FLOAT,
    low FLOAT,
    close_price FLOAT,
    adjclose FLOAT,
    volume BIGINT,
    event_time TIMESTAMP(3),
    WATERMARK FOR event_time as event_time
) WITH ( --connect to your kinesis data stream
    'connector' = 'kinesis', 
    'stream' = 'new-data-stream',
    'aws.region' = 'us-east-1',
    'scan.stream.initpos' = 'TRIM_HORIZON',
    'format' = 'json',
    'json.timestamp-format.standard' = 'ISO-8601'
);


%flink.pyflink

from pyflink.table import DataTypes
from pyflink.table.udf import udf
from datetime import datetime

START_PRICE = 92.30
START_YEAR = 2021
START_MONTH = 1


@udf(result_type=DataTypes.FLOAT(), input_types=[DataTypes.STRING(), DataTypes.FLOAT()])
def cmgr(date_str: str, price: float) -> float:
    try:
        # Extract year, month, day
        date_obj = datetime.strptime(date_str, "%m/%d/%Y")
        year, month, day = date_obj.year, date_obj.month, date_obj.day

        # Filter: not Jan 2021 and within first 3 days
        if (year == START_YEAR and month == START_MONTH) or day > 3:
            return 0.0

        # Months since Jan 2021
        num_months = (year - START_YEAR) * 12 + (month - START_MONTH)
        if num_months <= 0 or START_PRICE == 0:
            return 0.0

        growth_rate = ((price / START_PRICE) ** (1 / num_months) - 1) * 100
        return round(growth_rate, 6)
    except Exception:
        return 0.0

st_env.create_temporary_system_function("cmgr", cmgr)


%flink.ssql
SELECT event_time, cmgr(`date`, close_price) AS cmgr, `date`, close_price
FROM stock_table
WHERE cmgr(close_price, `date`) <> 0